In [4]:
import asyncio
import aiohttp
import json
from pprint import pprint
from attraction_api import fetch_attractions
from hotel_api import HotelRetriever
from flight_api import FlightRetriever
from llm_retrieve import llm_retrieve
from llm_generate import llm_generate

In [2]:
with open("user_inputs.json", "r") as f:
    inputs = json.load(f)

trip_plan = llm_retrieve(inputs)
print("Loaded cities and parameters:")
for city, info in trip_plan.items():
    print(f" • {city}: Arrival={info['Arrival Date']}, Departure={info['Departure Date']}, Airport={info['Airport']}")


Loaded cities and parameters:
 • Shimla: Arrival=2025-07-11, Departure=2025-07-13, Airport=SLV
 • Chandigarh: Arrival=2025-07-13, Departure=2025-07-14, Airport=IXC
 • Dehradun: Arrival=2025-07-14, Departure=2025-07-16, Airport=DED
 • Delhi: Arrival=2025-07-16, Departure=2025-07-17, Airport=DEL


In [5]:
async def gather_all():
    async with aiohttp.ClientSession() as session:
        attraction_tasks = [fetch_attractions(session, city) for city in trip_plan.keys()]
        attraction_results = await asyncio.gather(*attraction_tasks)

    hotel_results = HotelRetriever(trip_plan, numadults=inputs['num_people'])
    flight_results = FlightRetriever(trip_plan, numadults=inputs['num_people'])

    return attraction_results, hotel_results, flight_results


In [6]:
# Cell 4: Run & Merge
attractions, hotels, flights = await gather_all()    

[Attraction] 🔍 Fetching location_id for: Shimla
[Attraction] 🔍 Fetching location_id for: Chandigarh
[Attraction] 🔍 Fetching location_id for: Dehradun
[Attraction] 🔍 Fetching location_id for: Delhi
[Attraction] ✅ location_id for Shimla = 304552
[Attraction] 🔍 Fetching attractions for Shimla
[Attraction] ✅ location_id for Dehradun = 297687
[Attraction] 🔍 Fetching attractions for Dehradun
[Attraction] 🌐 Raw results count for Shimla: 20
[Attraction] 🎯 Returning 10 attractions for Shimla
[Attraction] 🌐 Raw results count for Dehradun: 20
[Attraction] 🎯 Returning 10 attractions for Dehradun
[Attraction] ✅ location_id for Chandigarh = 297596
[Attraction] 🔍 Fetching attractions for Chandigarh
[Attraction] ✅ location_id for Delhi = 304551
[Attraction] 🔍 Fetching attractions for Delhi
[Attraction] 🌐 Raw results count for Chandigarh: 20
[Attraction] 🎯 Returning 10 attractions for Chandigarh
[Attraction] 🌐 Raw results count for Delhi: 20
[Attraction] 🎯 Returning 10 attractions for Delhi


TypeError: string indices must be integers, not 'str'

In [12]:
# Merge results by city
combined = {city: {"city": city, "attractions": [], "hotels": [], "flights": []} for city in trip_plan}

# Add attractions
for entry in attractions:
    combined[entry["city"]]["attractions"] = entry["attractions"]

# Add hotels
for city, hotel_data in hotels.items():
    combined[city]["hotels"] = hotel_data

# Add flights
for entry in flights:
    from_city = entry["from"]
    to_city = entry["to"]
    date = entry["date"]
    flights_data = entry.get("flights")
    note = entry.get("note")

    # Associate the flight with departure city
    combined[from_city].setdefault("flights", []).append({
        "to": to_city,
        "date": date,
        "flights": flights_data,
        "note": note
    })

In [13]:
final_list = list(combined.values())

print("\n\u2705 After merging, sample output for first city:")
print(json.dumps(final_list[0], indent=2))


✅ After merging, sample output for first city:
{
  "city": "Shimla",
  "attractions": [
    {
      "name": "Jakhu Hanuman Temple",
      "num_reviews": "2091",
      "rating": "4.4",
      "ranking_position": "3",
      "neighborhood_info": null,
      "description": "",
      "subcategory": [
        {
          "key": "47",
          "name": "Sights & Landmarks"
        }
      ]
    },
    {
      "name": "The Mall",
      "num_reviews": "2932",
      "rating": "4.2",
      "ranking_position": "7",
      "neighborhood_info": null,
      "description": "",
      "subcategory": [
        {
          "key": "47",
          "name": "Sights & Landmarks"
        }
      ]
    },
    {
      "name": "Kalka - Shimla Railway",
      "num_reviews": "1826",
      "rating": "4.3",
      "ranking_position": "9",
      "neighborhood_info": null,
      "description": "This railway was built in 1903 to connect the summer capital of India to Punjab.",
      "subcategory": [
        {
          "ke

In [14]:
import pandas as pd
from IPython.display import display

# Flatten into a DataFrame (one row per city)
df = pd.json_normalize(final_list, sep="_")
display(df)

# Save JSON
out_path = "combined_data_hotel_attraction_flight_final.json"
with open(out_path, "w") as f:
    json.dump(final_list, f, indent=2)

print("\u2705 Saved combined results to", out_path)

,city,attractions,flights,hotels_bayesian_review_score,hotels_popularity,hotels_price
0,Shimla,"[{'name': 'Jakhu Hanuman Temple', 'num_reviews...","[{'to': 'Chandigarh', 'date': '2025-07-13', 'f...","[{'name': 'Wildflower Hall, An Oberoi Resort, ...",[{'name': 'Himalayan Mountain and Valley View ...,"[{'name': 'Staynest Mashobra with balcony', 'p..."
1,Chandigarh,"[{'name': 'The Rock Garden of Chandigarh', 'nu...","[{'to': 'Dehradun', 'date': '2025-07-14', 'fli...","[{'name': 'Anchorage 42', 'price': 282.0490607...",[{'name': 'Hotel The Pearl Grand - Top Rated !...,"[{'name': 'Hotel Golden Height2', 'price': 12...."
2,Dehradun,"[{'name': 'Forest Research Institute', 'num_re...","[{'to': 'Delhi', 'date': '2025-07-16', 'flight...","[{'name': 'vera home stay', 'price': 97.081866...","[{'name': 'Stayble Homestay', 'price': 71.1977...","[{'name': 'King Queen Home', 'price': 41.96035..."
3,Delhi,"[{'name': 'Qutub Minar', 'num_reviews': '13493...",[],"[{'name': 'Tatvamasi Homestay', 'price': 111.6...",[{'name': 'Hotel Namaste Hindustan Just a Minu...,"[{'name': 'Hotel Indo continental', 'price': 9..."


✅ Saved combined results to combined_data_hotel_attraction_flight_final.json


In [15]:
# Generate the final itinerary to be displayed
narrative = llm_generate(inputs, trip_plan, final_list)

In [18]:
with open("generated_itinerary_short.txt", "w") as f:
    f.write(narrative)

## only testing llm_retrieve markdown

In [1]:
from llm_generate import llm_generate

In [12]:
with open("user_inputs.json", "r") as f:
    inputs = json.load(f)

In [13]:
inputs

{'country': 'United States',
 'budget_in_usd': 2000.0,
 'from_date': '2025-07-11',
 'to_date': '2025-07-17',
 'num_cities': '3 to 5',
 'num_people': 4,
 'user_pref': 'I want to go to beaches and casinos.'}

In [14]:
with open("itinerary.json", "r") as f:
    trip_plan = json.load(f)

In [15]:
trip_plan

{'Miami': {'Arrival Date': '2025-07-11',
  'Departure Date': '2025-07-13',
  'Airport': 'MIA'},
 'Atlantic City': {'Arrival Date': '2025-07-13',
  'Departure Date': '2025-07-15',
  'Airport': 'ACY'},
 'Las Vegas': {'Arrival Date': '2025-07-15',
  'Departure Date': '2025-07-17',
  'Airport': 'LAS'}}

In [16]:
with open("combined_data_hotel_attraction_flight_final.json", "r") as f:
    final_list = json.load(f)

In [17]:
final_itinerary = llm_generate(inputs, trip_plan, final_list)

In [18]:
with open("final_itinerary_output23rdApril_beaches.md", "w", encoding="utf-8") as f:
    f.write(final_itinerary)